# Amazon SageMaker Batch Transform: Associate prediction results with their corresponding input records


_**Use SageMaker's XGBoost to train a binary classification model and for a list of tumors in batch file, predict if each is malignant**_

_**It also shows how to use the input output joining / filter feature in Batch transform in details**_

---



## Background
This purpose of this notebook is to train a model using SageMaker's XGBoost and UCI's breast cancer diagnostic data set to illustrate at how to run batch inferences and how to use the Batch Transform I/O join feature. UCI's breast cancer diagnostic data set is available at https://archive.ics.uci.edu/ml/datasets/Breast+Cancer+Wisconsin+%28Diagnostic%29. The data set is also available on Kaggle at https://www.kaggle.com/uciml/breast-cancer-wisconsin-data. The purpose here is to use this data set to build a predictve model of whether a breast mass image indicates benign or malignant tumor. 



---

## Setup

Let's start by specifying:

* The SageMaker role arn used to give training and batch transform access to your data. The snippet below will use the same role used by your SageMaker notebook instance. Otherwise, specify the full ARN of a role with the SageMakerFullAccess policy attached.
* The S3 bucket that you want to use for training and storing model objects.

In [2]:
!pip3 install -U sagemaker

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 10.3 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: sagemaker
    Found existing installation: sagemaker 2.215.0
    Uninstalling sagemaker-2.215.0:
      Successfully uninstalled sagemaker-2.215.0


In [3]:
import os
import boto3
import sagemaker

role = sagemaker.get_execution_role()
sess = sagemaker.Session()
region = sess.boto_region_name

bucket = sess.default_bucket()
prefix = "DEMO-breast-cancer-prediction-xgboost-highlevel"

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml


---
## Data sources

> Dua, D. and Graff, C. (2019). UCI Machine Learning Repository [http://archive.ics.uci.edu/ml]. Irvine, CA: University of California, School of Information and Computer Science.

> Breast Cancer Wisconsin (Diagnostic) Data Set [https://archive.ics.uci.edu/ml/datasets/Breast+Cancer+Wisconsin+(Diagnostic)].

> _Also see:_ Breast Cancer Wisconsin (Diagnostic) Data Set [https://www.kaggle.com/uciml/breast-cancer-wisconsin-data].

## Data preparation


Let's download the data and save it in the local folder with the name data.csv and take a look at it.

In [4]:
import pandas as pd
import numpy as np

s3 = boto3.client("s3")

filename = "wdbc.csv"
s3.download_file(
    f"sagemaker-example-files-prod-{region}", "datasets/tabular/breast_cancer/wdbc.csv", filename
)
data = pd.read_csv(filename, header=None)

# specify columns extracted from wbdc.names
data.columns = [
    "id",
    "diagnosis",
    "radius_mean",
    "texture_mean",
    "perimeter_mean",
    "area_mean",
    "smoothness_mean",
    "compactness_mean",
    "concavity_mean",
    "concave points_mean",
    "symmetry_mean",
    "fractal_dimension_mean",
    "radius_se",
    "texture_se",
    "perimeter_se",
    "area_se",
    "smoothness_se",
    "compactness_se",
    "concavity_se",
    "concave points_se",
    "symmetry_se",
    "fractal_dimension_se",
    "radius_worst",
    "texture_worst",
    "perimeter_worst",
    "area_worst",
    "smoothness_worst",
    "compactness_worst",
    "concavity_worst",
    "concave points_worst",
    "symmetry_worst",
    "fractal_dimension_worst",
]

# save the data
data.to_csv("data.csv", sep=",", index=False)

data.sample(8)

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
24,852552,M,16.650,21.38,110.00,904.6,0.11210,0.14570,0.15250,0.09170,...,26.46,31.56,177.00,2215.0,0.1805,0.3578,0.4695,0.20950,0.3613,0.09564
359,901034301,B,9.436,18.32,59.82,278.6,0.10090,0.05956,0.02710,0.01406,...,12.02,25.02,75.79,439.6,0.1333,0.1049,0.1144,0.05052,0.2454,0.08136
483,912558,B,13.700,17.64,87.76,571.1,0.09950,0.07957,0.04548,0.03160,...,14.96,23.53,95.78,686.5,0.1199,0.1346,0.1742,0.09077,0.2518,0.06960
481,91227,B,13.900,19.24,88.73,602.9,0.07991,0.05326,0.02995,0.02070,...,16.41,26.42,104.40,830.5,0.1064,0.1415,0.1673,0.08150,0.2356,0.07603
44,85638502,M,13.170,21.81,85.42,531.5,0.09714,0.10470,0.08259,0.05252,...,16.23,29.89,105.50,740.7,0.1503,0.3904,0.3728,0.16070,0.3693,0.09618
283,8912280,M,16.240,18.77,108.80,805.1,0.10660,0.18020,0.19480,0.09052,...,18.55,25.09,126.90,1031.0,0.1365,0.4706,0.5026,0.17320,0.2770,0.10630
435,908489,M,13.980,19.62,91.12,599.5,0.10600,0.11330,0.11260,0.06463,...,17.04,30.80,113.90,869.3,0.1613,0.3568,0.4069,0.18270,0.3179,0.10550
18,849014,M,19.810,22.15,130.00,1260.0,0.09831,0.10270,0.14790,0.09498,...,27.32,30.88,186.80,2398.0,0.1512,0.3150,0.5372,0.23880,0.2768,0.07615


#### Key observations:
* The data has 569 observations and 32 columns.
* The first field is the 'id' attribute that we will want to drop before batch inference and add to the final inference output next to the probability of malignancy.
* Second field, 'diagnosis', is an indicator of the actual diagnosis ('M' = Malignant; 'B' = Benign).
* There are 30 other numeric features that we will use for training and inferencing.

Let's replace the M/B diagnosis with a 1/0 boolean value. 

In [5]:
data["diagnosis"] = data["diagnosis"].apply(lambda x: ((x == "M")) + 0)
data.sample(8)

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
143,869224,0,12.90,15.92,83.74,512.2,0.08677,0.09509,0.04894,0.03088,...,14.48,21.82,97.17,643.8,0.1312,0.2548,0.2090,0.10120,0.3549,0.08118
93,862009,0,13.45,18.30,86.60,555.1,0.10220,0.08165,0.03974,0.02780,...,15.10,25.94,97.59,699.4,0.1339,0.1751,0.1381,0.07911,0.2678,0.06603
118,864877,1,15.78,22.91,105.70,782.6,0.11550,0.17520,0.21330,0.09479,...,20.19,30.50,130.30,1272.0,0.1855,0.4925,0.7356,0.20340,0.3274,0.12520
379,9013838,1,11.08,18.83,73.30,361.6,0.12160,0.21540,0.16890,0.06367,...,13.24,32.82,91.76,508.1,0.2184,0.9379,0.8402,0.25240,0.4154,0.14030
441,909445,1,17.27,25.42,112.40,928.8,0.08331,0.11090,0.12040,0.05736,...,20.38,35.46,132.80,1284.0,0.1436,0.4122,0.5036,0.17390,0.2500,0.07944
43,856106,1,13.28,20.28,87.32,545.2,0.10410,0.14360,0.09847,0.06158,...,17.38,28.00,113.10,907.2,0.1530,0.3724,0.3664,0.14920,0.3739,0.10270
363,9010872,0,16.50,18.29,106.60,838.1,0.09686,0.08468,0.05862,0.04835,...,18.13,25.45,117.20,1009.0,0.1338,0.1679,0.1663,0.09123,0.2394,0.06469
519,917080,0,12.75,16.70,82.51,493.8,0.11250,0.11170,0.03880,0.02995,...,14.45,21.74,93.63,624.1,0.1475,0.1979,0.1423,0.08045,0.3071,0.08557


Let's split the data as follows: 80% for training, 10% for validation and let's set 10% aside for our batch inference job. In addition, let's drop the 'id' field on the training set and validation set as 'id' is not a training feature. For our batch set however, we keep the 'id' feature. We'll want to filter it out prior to running our inferences so that the input data features match the ones of training set and then ultimately, we'll want to join it with inference result. We are however dropping the diagnosis attribute for the batch set since this is what we'll try to predict.

In [6]:
# data split in three sets, training, validation and batch inference
rand_split = np.random.rand(len(data))
train_list = rand_split < 0.8
val_list = (rand_split >= 0.8) & (rand_split < 0.9)
batch_list = rand_split >= 0.9

data_train = data[train_list].drop(["id"], axis=1)
data_val = data[val_list].drop(["id"], axis=1)
data_batch = data[batch_list].drop(["diagnosis"], axis=1)
data_batch_noID = data_batch.drop(["id"], axis=1)

Let's upload those data sets in S3

In [7]:
train_file = "train_data.csv"
data_train.to_csv(train_file, index=False, header=False)
sess.upload_data(train_file, key_prefix="{}/train".format(prefix))

validation_file = "validation_data.csv"
data_val.to_csv(validation_file, index=False, header=False)
sess.upload_data(validation_file, key_prefix="{}/validation".format(prefix))

batch_file = "batch_data.csv"
data_batch.to_csv(batch_file, index=False, header=False)
sess.upload_data(batch_file, key_prefix="{}/batch".format(prefix))

batch_file_noID = "batch_data_noID.csv"
data_batch_noID.to_csv(batch_file_noID, index=False, header=False)
sess.upload_data(batch_file_noID, key_prefix="{}/batch".format(prefix))

's3://sagemaker-us-east-1-904981812149/DEMO-breast-cancer-prediction-xgboost-highlevel/batch/batch_data_noID.csv'

---

## Training job and model creation

The below cell uses the [SageMaker Python SDK](https://github.com/aws/sagemaker-python-sdk) to kick off the training job using both our training set and validation set. Not that the objective is set to 'binary:logistic' which trains a model to output a probability between 0 and 1 (here the probability of a tumor being malignant).

In [8]:
%%time
from time import gmtime, strftime

job_name = "xgb-" + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
output_location = "s3://{}/{}/output/{}".format(bucket, prefix, job_name)
image = sagemaker.image_uris.retrieve(
    framework="xgboost", region=boto3.Session().region_name, version="1.7-1"
)

sm_estimator = sagemaker.estimator.Estimator(
    image,
    role,
    instance_count=1,
    instance_type="ml.m5.xlarge",
    volume_size=50,
    input_mode="File",
    output_path=output_location,
    sagemaker_session=sess,
)

sm_estimator.set_hyperparameters(
    objective="binary:logistic",
    max_depth=5,
    eta=0.2,
    gamma=4,
    min_child_weight=6,
    subsample=0.8,
    verbosity=0,
    num_round=100,
)

train_data = sagemaker.inputs.TrainingInput(
    "s3://{}/{}/train".format(bucket, prefix),
    distribution="FullyReplicated",
    content_type="text/csv",
    s3_data_type="S3Prefix",
)
validation_data = sagemaker.inputs.TrainingInput(
    "s3://{}/{}/validation".format(bucket, prefix),
    distribution="FullyReplicated",
    content_type="text/csv",
    s3_data_type="S3Prefix",
)
data_channels = {"train": train_data, "validation": validation_data}

# Start training by calling the fit method in the estimator
sm_estimator.fit(inputs=data_channels, job_name=job_name, logs=True)

INFO:sagemaker:Creating training-job with name: xgb-2024-06-01-19-12-53


2024-06-01 19:12:53 Starting - Starting the training job...
2024-06-01 19:13:08 Starting - Preparing the instances for training...
2024-06-01 19:13:33 Downloading - Downloading input data...
2024-06-01 19:13:58 Downloading - Downloading the training image...
2024-06-01 19:14:54 Training - Training image download completed. Training in progress.
2024-06-01 19:14:54 Uploading - Uploading generated training model[2024-06-01 19:14:46.661 ip-10-0-234-149.ec2.internal:7 INFO utils.py:28] RULE_JOB_STOP_SIGNAL_FILENAME: None
[2024-06-01 19:14:46.683 ip-10-0-234-149.ec2.internal:7 INFO profiler_config_parser.py:111] User has disabled profiler.
[2024-06-01:19:14:47:INFO] Imported framework sagemaker_xgboost_container.training
[2024-06-01:19:14:47:INFO] Failed to parse hyperparameter objective value binary:logistic to Json.
Returning the value itself
[2024-06-01:19:14:47:INFO] No GPUs detected (normal if no gpus installed)
[2024-06-01:19:14:47:INFO] Running XGBoost Sagemaker in algorithm mode
[20

---

## Batch Transform

In SageMaker Batch Transform, we introduced 3 new attributes - __input_filter__, __join_source__ and __output_filter__. In the below cell, we use the [SageMaker Python SDK](https://github.com/aws/sagemaker-python-sdk) to kick-off several Batch Transform jobs using different configurations of these 3 new attributes. Please refer to [this page](https://docs.aws.amazon.com/sagemaker/latest/dg/batch-transform-data-processing.html) to learn more about how to use them.




#### 1. Create a transform job with the default configurations
Let's first skip these 3 new attributes and inspect the inference results. We'll use it as a baseline to compare to the results with data processing.

In [9]:
%%time

sm_transformer = sm_estimator.transformer(1, "ml.m4.xlarge")

# start a transform job
input_location = "s3://{}/{}/batch/{}".format(
    bucket, prefix, batch_file_noID
)  # use input data without ID column
sm_transformer.transform(input_location, content_type="text/csv", split_type="Line")
sm_transformer.wait()

INFO:sagemaker:Creating model with name: sagemaker-xgboost-2024-06-01-19-15-35-800
INFO:sagemaker:Creating transform job with name: sagemaker-xgboost-2024-06-01-19-15-36-507


........................................[2024-06-01:19:22:15:INFO] No GPUs detected (normal if no gpus installed)
[2024-06-01:19:22:15:INFO] No GPUs detected (normal if no gpus installed)
[2024-06-01:19:22:15:INFO] nginx config: 
worker_processes auto;
daemon off;
pid /tmp/nginx.pid;
error_log  /dev/stderr;
worker_rlimit_nofile 4096;
events {
  worker_connections 2048;
}
[2024-06-01:19:22:15:INFO] No GPUs detected (normal if no gpus installed)
[2024-06-01:19:22:15:INFO] No GPUs detected (normal if no gpus installed)
[2024-06-01:19:22:15:INFO] nginx config: 
worker_processes auto;
daemon off;
pid /tmp/nginx.pid;
error_log  /dev/stderr;
worker_rlimit_nofile 4096;
events {
  worker_connections 2048;
}
http {
  include /etc/nginx/mime.types;
  default_type application/octet-stream;
  access_log /dev/stdout combined;
  upstream gunicorn {
    server unix:/tmp/gunicorn.sock;
  }
  server {
    listen 8080 deferred;
    client_max_body_size 0;
    keepalive_timeout 3;
    location ~ ^/(ping|i

Let's inspect the output of the Batch Transform job in S3. It should show the list probabilities of tumors being malignant.

In [10]:
import re


def get_csv_output_from_s3(s3uri, batch_file):
    file_name = "{}.out".format(batch_file)
    match = re.match("s3://([^/]+)/(.*)", "{}/{}".format(s3uri, file_name))
    output_bucket, output_prefix = match.group(1), match.group(2)
    s3.download_file(output_bucket, output_prefix, file_name)
    return pd.read_csv(file_name, sep=",", header=None)

In [11]:
output_df = get_csv_output_from_s3(sm_transformer.output_path, batch_file_noID)
output_df.head(8)

,0
0,0.951907
1,0.990928
2,0.986776
3,0.994184
4,0.923253
5,0.009414
6,0.005347
7,0.951907


#### 2. Join the input and the prediction results 
Now, let's associate the prediction results with their corresponding input records. We can also use the __input_filter__ to exclude the ID column easily and there's no need to have a separate file in S3.

* Set __input_filter__ to "$[1:]": indicates that we are excluding column 0 (the 'ID') before processing the inferences and keeping everything from column 1 to the last column (all the features or predictors)  
  
  
* Set __join_source__ to "Input": indicates our desire to join the input data with the inference results  

* Leave __output_filter__ to default ('$'), indicating that the joined input and inference results be will saved as output.

In [12]:
# content_type / accept and split_type / assemble_with are required to use IO joining feature
sm_transformer.assemble_with = "Line"
sm_transformer.accept = "text/csv"

# start a transform job
input_location = "s3://{}/{}/batch/{}".format(
    bucket, prefix, batch_file
)  # use input data with ID column cause InputFilter will filter it out
sm_transformer.transform(
    input_location,
    split_type="Line",
    content_type="text/csv",
    input_filter="$[1:]",
    join_source="Input",
)
sm_transformer.wait()

INFO:sagemaker:Creating transform job with name: sagemaker-xgboost-2024-06-01-19-23-11-624


.........................................[2024-06-01:19:30:02:INFO] No GPUs detected (normal if no gpus installed)
[2024-06-01:19:30:02:INFO] No GPUs detected (normal if no gpus installed)
[2024-06-01:19:30:02:INFO] nginx config: 
worker_processes auto;
daemon off;
pid /tmp/nginx.pid;
error_log  /dev/stderr;
worker_rlimit_nofile 4096;
events {
  worker_connections 2048;
}
http {
  include /etc/nginx/mime.types;
  default_type application/octet-stream;
  access_log /dev/stdout combined;
  upstream gunicorn {
    server unix:/tmp/gunicorn.sock;
  }
  server {
    listen 8080 deferred;
    client_max_body_size 0;
    keepalive_timeout 3;
    location ~ ^/(ping|invocations|execution-parameters) {
      proxy_set_header X-Forwarded-For $proxy_add_x_forwarded_for;
      proxy_set_header Host $http_host;
      proxy_redirect off;
      proxy_read_timeout 60s;
      proxy_pass http://gunicorn;
    }
    location / {
      return 404 "{}";
    }
  }
}
[2024-06-01 19:30:02 +0000] [19] [INFO] Sta

Let's inspect the output of the Batch Transform job in S3. It should show the list of tumors identified by their original feature columns and their corresponding probabilities of being malignant.

In [13]:
output_df = get_csv_output_from_s3(sm_transformer.output_path, batch_file)
output_df.head(8)

,0,1,2,3,4,5,6,7,8,9,...,22,23,24,25,26,27,28,29,30,31
0,842302,17.990,10.38,122.80,1001.0,0.11840,0.27760,0.30010,0.14710,0.2419,...,17.33,184.60,2019.0,0.1622,0.66560,0.7119,0.26540,0.4601,0.11890,0.951907
1,84300903,19.690,21.25,130.00,1203.0,0.10960,0.15990,0.19740,0.12790,0.2069,...,25.53,152.50,1709.0,0.1444,0.42450,0.4504,0.24300,0.3613,0.08758,0.990928
2,852781,18.610,20.25,122.10,1094.0,0.09440,0.10660,0.14900,0.07731,0.1697,...,27.26,139.90,1403.0,0.1338,0.21170,0.3446,0.14900,0.2341,0.07421,0.986776
3,854002,19.270,26.47,127.90,1162.0,0.09401,0.17190,0.16570,0.07593,0.1853,...,30.90,161.40,1813.0,0.1509,0.65900,0.6091,0.17850,0.3672,0.11230,0.994184
4,854268,14.250,21.72,93.63,633.0,0.09823,0.10980,0.13190,0.05598,0.1885,...,30.36,116.20,799.6,0.1446,0.42380,0.5186,0.14470,0.3591,0.10140,0.923253
5,859465,11.310,19.04,71.80,394.1,0.08139,0.04701,0.03709,0.02230,0.1516,...,23.84,78.00,466.7,0.1290,0.09148,0.1444,0.06961,0.2400,0.06641,0.009414
6,859711,8.888,14.64,58.79,244.0,0.09783,0.15310,0.08606,0.02872,0.1902,...,15.67,62.56,284.4,0.1207,0.24360,0.1434,0.04786,0.2254,0.10840,0.005347
7,8610637,18.050,16.15,120.20,1006.0,0.10650,0.21460,0.16840,0.10800,0.2152,...,18.91,150.10,1610.0,0.1478,0.56340,0.3786,0.21020,0.3751,0.11080,0.951907


#### 3. Update the output filter to keep only ID and prediction results
Let's change __output_filter__ to "$[0,-1]", indicating that when presenting the output, we only want to keep column 0 (the 'ID') and the last column (the inference result i.e. the probability of a given tumor to be malignant)

In [14]:
# start another transform job
sm_transformer.transform(
    input_location,
    split_type="Line",
    content_type="text/csv",
    input_filter="$[1:]",
    join_source="Input",
    output_filter="$[0,-1]",
)
sm_transformer.wait()

INFO:sagemaker:Creating transform job with name: sagemaker-xgboost-2024-06-01-19-30-46-282


.........................................[2024-06-01:19:37:32:INFO] No GPUs detected (normal if no gpus installed)
[2024-06-01:19:37:32:INFO] No GPUs detected (normal if no gpus installed)
[2024-06-01:19:37:32:INFO] No GPUs detected (normal if no gpus installed)
[2024-06-01:19:37:32:INFO] nginx config: 
worker_processes auto;
daemon off;
pid /tmp/nginx.pid;
error_log  /dev/stderr;
worker_rlimit_nofile 4096;
events {
  worker_connections 2048;
}
http {
  include /etc/nginx/mime.types;
  default_type application/octet-stream;
  access_log /dev/stdout combined;
  upstream gunicorn {
    server unix:/tmp/gunicorn.sock;
  }
  server {
    listen 8080 deferred;
    client_max_body_size 0;
    keepalive_timeout 3;
    location ~ ^/(ping|invocations|execution-parameters) {
      proxy_set_header X-Forwarded-For $proxy_add_x_forwarded_for;
      proxy_set_header Host $http_host;
      proxy_redirect off;
      proxy_read_timeout 60s;
      proxy_pass http://gunicorn;
    }
    location / {
    

Now, let's inspect the output of the Batch Transform job in S3 again. It should show 2 columns: the ID and their corresponding probabilities of being malignant.

In [15]:
output_df = get_csv_output_from_s3(sm_transformer.output_path, batch_file)
output_df.head(8)

,0,1
0,842302,0.951907
1,84300903,0.990928
2,852781,0.986776
3,854002,0.994184
4,854268,0.923253
5,859465,0.009414
6,859711,0.005347
7,8610637,0.951907


create_model(role=role, image_uri=XGBOOST_IMAGE)In summary, we can use newly introduced 3 attributes - __input_filter__, __join_source__, __output_filter__ to 
1. Filter / select useful features from the input dataset. e.g. exclude ID columns.
2. Associate the prediction results with their corresponding input records.
3. Filter the original or joined results before saving to S3. e.g. keep ID and probability columns only.

## Upload the Sagemaker Model created during our training job to the Sagemaker Model Registry

In [16]:
sagemaker = boto3.client("sagemaker")

#model_name = "sagemaker-xgboost-2023-08-22-05-28-37-903"
model_name = job_name
print(model_name)


info = sagemaker.describe_training_job(TrainingJobName=model_name)
model_data = info["ModelArtifacts"]["S3ModelArtifacts"]

primary_container = {"Image": image, "ModelDataUrl": model_data}

# Save our model to the Sagemaker Model Registry
create_model_response = sagemaker.create_model(
    ModelName=model_name, ExecutionRoleArn=role, PrimaryContainer=primary_container
)

print(create_model_response["ModelArn"])

xgb-2024-06-01-19-12-53
arn:aws:sagemaker:us-east-1:904981812149:model/xgb-2024-06-01-19-12-53


In [17]:
# Inspect Training Job Details
info

{'TrainingJobName': 'xgb-2024-06-01-19-12-53',
 'TrainingJobArn': 'arn:aws:sagemaker:us-east-1:904981812149:training-job/xgb-2024-06-01-19-12-53',
 'ModelArtifacts': {'S3ModelArtifacts': 's3://sagemaker-us-east-1-904981812149/DEMO-breast-cancer-prediction-xgboost-highlevel/output/xgb-2024-06-01-19-12-53/xgb-2024-06-01-19-12-53/output/model.tar.gz'},
 'TrainingJobStatus': 'Completed',
 'SecondaryStatus': 'Completed',
 'HyperParameters': {'eta': '0.2',
  'gamma': '4',
  'max_depth': '5',
  'min_child_weight': '6',
  'num_round': '100',
  'objective': 'binary:logistic',
  'subsample': '0.8',
  'verbosity': '0'},
 'AlgorithmSpecification': {'TrainingImage': '683313688378.dkr.ecr.us-east-1.amazonaws.com/sagemaker-xgboost:1.7-1',
  'TrainingInputMode': 'File',
  'MetricDefinitions': [{'Name': 'train:mae',
    'Regex': '.*\\[[0-9]+\\].*#011train-mae:([-+]?[0-9]*\\.?[0-9]+(?:[eE][-+]?[0-9]+)?).*'},
   {'Name': 'validation:aucpr',
    'Regex': '.*\\[[0-9]+\\].*#011validation-aucpr:([-+]?[0-9]*\

## Create Endpoint Configuration

In [18]:
# Create Endpoint Configuration


# Create an endpoint config name. Here we create one based on the date  
# so it we can search endpoints based on creation time.
endpoint_config_name = 'lab4-1-endpoint-config' + strftime("%Y-%m-%d-%H-%M-%S", gmtime())                            
                            
instance_type = 'ml.m5.xlarge'

endpoint_config_response = sagemaker.create_endpoint_config(
    EndpointConfigName=endpoint_config_name, # You will specify this name in a CreateEndpoint request.
    # List of ProductionVariant objects, one for each model that you want to host at this endpoint.
    ProductionVariants=[
        {
            "VariantName": "variant1", # The name of the production variant.
            "ModelName": model_name, 
            "InstanceType": instance_type, # Specify the compute instance type.
            "InitialInstanceCount": 1 # Number of instances to launch initially.
        }
    ]
)

print(f"Created EndpointConfig: {endpoint_config_response['EndpointConfigArn']}")


Created EndpointConfig: arn:aws:sagemaker:us-east-1:904981812149:endpoint-config/lab4-1-endpoint-config2024-06-01-19-38-22


In [19]:
# Deploy our model to real-time endpoint

endpoint_name = 'lab4-1-endpoint' + strftime("%Y-%m-%d-%H-%M-%S", gmtime())                            


create_endpoint_response = sagemaker.create_endpoint(
                                            EndpointName=endpoint_name, 
                                            EndpointConfigName=endpoint_config_name) 

In [21]:
# Wait for endpoint to spin up
import time 

sagemaker.describe_endpoint(EndpointName=endpoint_name)

while True:
    print("Getting Job Status")
    res = sagemaker.describe_endpoint(EndpointName=endpoint_name)
    state = res["EndpointStatus"]
    
    if state == "InService":
        print("Endpoint in Service")
        break
    elif state == "Creating":
        print("Endpoint still creating...")
        time.sleep(60)
    else:
        print("Endpoint Creation Error - Check Sagemaker Console")
        break

Getting Job Status
Endpoint still creating...
Getting Job Status
Endpoint in Service


In [22]:
# Invoke Endpoint

sagemaker_runtime = boto3.client("sagemaker-runtime", region_name=region)

response = sagemaker_runtime.invoke_endpoint(
                            EndpointName=endpoint_name,
                            ContentType='text/csv',
                            Body=data_batch_noID.to_csv(header=None, index=False).strip('\n').split('\n')[0]
                            )
print(response['Body'].read().decode('utf-8'))

0.9519073367118835



In [23]:
# Examine Response Body

response

{'ResponseMetadata': {'RequestId': '0eb25020-0f1e-46f2-8d06-7edcbff609e6',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '0eb25020-0f1e-46f2-8d06-7edcbff609e6',
   'x-amzn-invoked-production-variant': 'variant1',
   'date': 'Sat, 01 Jun 2024 19:42:27 GMT',
   'content-type': 'text/csv; charset=utf-8',
   'content-length': '19',
   'connection': 'keep-alive'},
  'RetryAttempts': 0},
 'ContentType': 'text/csv; charset=utf-8',
 'InvokedProductionVariant': 'variant1',
 'Body': <botocore.response.StreamingBody at 0x7f3ab6e7c910>}

In [ ]:
# Delete Endpoint

#sagemaker.delete_endpoint(EndpointName=endpoint_name)

# Part 1: Set Up Model Group


First, import the required packages and set up the SageMaker Boto3 client.

In [28]:
import time 
import os
from sagemaker import get_execution_role, session
import boto3

#region = boto3.Session().region_name()
#role = get_execution_role()

sm_client = boto3.client('sagemaker', region_name= region)

Now create the Model Group.

In [31]:
import time

model_package_group_name = "xgboost-breast-cancer-detector-" + str(round(time.time()))
model_package_group_input_dict = {
    "ModelPackageGroupName" : model_package_group_name, 
    "ModelPackageGroupDescription" : "Group for versioned XGBoost models for breast cancer detection."
    }
create_model_package_group_response = sm_client.create_model_package_group(**model_package_group_input_dict)
print("ModelPackageGroup Arn : {}".format(create_model_package_group_response['ModelPackageGroupArn']))

ModelPackageGroup Arn : arn:aws:sagemaker:us-east-1:904981812149:model-package-group/xgboost-breast-cancer-detector-1717272235


Get the description of the model group

In [32]:
response = sm_client.describe_model_package_group(ModelPackageGroupName=model_package_group_name)
print(response)


{'ModelPackageGroupName': 'xgboost-breast-cancer-detector-1717272235', 'ModelPackageGroupArn': 'arn:aws:sagemaker:us-east-1:904981812149:model-package-group/xgboost-breast-cancer-detector-1717272235', 'ModelPackageGroupDescription': 'Group for versioned XGBoost models for breast cancer detection.', 'CreationTime': datetime.datetime(2024, 6, 1, 20, 3, 55, 410000, tzinfo=tzlocal()), 'CreatedBy': {'UserProfileArn': 'arn:aws:sagemaker:us-east-1:904981812149:user-profile/d-bxoyca62lhab/shyam', 'UserProfileName': 'shyam', 'DomainId': 'd-bxoyca62lhab', 'IamIdentity': {'Arn': 'arn:aws:sts::904981812149:assumed-role/LabRole/SageMaker', 'PrincipalId': 'AROA5FNI6SO26R3SUIJA5:SageMaker'}}, 'ModelPackageGroupStatus': 'Completed', 'ResponseMetadata': {'RequestId': 'fb183ab7-543a-4f46-9e43-0aa2e4c8a45b', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': 'fb183ab7-543a-4f46-9e43-0aa2e4c8a45b', 'content-type': 'application/x-amz-json-1.1', 'content-length': '654', 'date': 'Sat, 01 Jun 2024 20:

# Part 2: Set Up Model Package

First, set up the parameter dictionary to pass to the create_model_package API operation.

In [ ]:
# Specify the model source

# model_url is a ModelArtifacts or S3ModelArtifacts
model_url = "s3://sagemaker-us-east-1-904981812149/DEMO-breast-cancer-prediction-xgboost-highlevel/output/xgb-2024-06-01-19-12-53/xgb-2024-06-01-19-12-53/output/model.tar.gz"
image_uri = ""

modelpackage_inference_specification =  {
    "InferenceSpecification": {
      "Containers": [
         {
            "Image": image_uri,
	    "ModelDataUrl": model_url
         }
      ],
      "SupportedContentTypes": [ "text/csv" ],
      "SupportedResponseMIMETypes": [ "text/csv" ],
   }
 }

create_model_package_input_dict = {
    "ModelPackageGroupName" : model_package_group_name,
    "ModelPackageDescription" : "XGBoost model trained to predict breast cancer (Benign and Malignant)",
    "ModelApprovalStatus" : "PendingManualApproval"
}
create_model_package_input_dict.update(modelpackage_inference_specification)

create_model_package_input_dict

Then call the create_model_package API operation, passing in the parameter dictionary that just set up.

In [ ]:
create_model_package_response = sm_client.create_model_package(**create_model_package_input_dict)
model_package_arn = create_model_package_response["ModelPackageArn"]
print('ModelPackage Version ARN : {}'.format(model_package_arn))

Running describe model package using boto3

In [ ]:
response = sm_client.describe_model_package(
    ModelPackageName = model_package_arn
    )
print(response)

# Part 3: Write the Model Card